In [16]:
import time

# get the start time
st = time.time()


import numpy as np  
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import sqlalchemy as sa
import json 

"""
    Arguement: search_key - String with the word that the user wants to do the search
    Return argument: ids - a list of project id's that need will be displayed to user
"""
def semantic_search(search_key):
    #search_key = 'Science Competition'    
    
    #threshold = 0.25
    #path_embeddings = "/home/claudia/Documents/women++/deploy-impact-22-openedu-c/NLP/embeddings.npy"
    #path_ids = "/home/claudia/Documents/women++/deploy-impact-22-openedu-c/NLP/ids.npy"

    
    # loading the embeddings and the ids from binary file
    ##project_embeddings = np.load(path_embeddings, allow_pickle=True)
    ##ids = np.load(path_ids, allow_pickle=True)
    
    
    engine = sa.create_engine('postgresql://django@openeduc-db:deploy-impact-2022@openeduc-db.postgres.database.azure.com:5432/openeduc-db', connect_args={"sslmode": "require"})
    df_id = pd.read_sql_query('SELECT id, title FROM edu_data_edumaterial', con=engine) #without title the ids were sorted.
    ids = df_id.drop(['title'], axis=1).to_numpy()
    df_embeddings = pd.read_sql_query('SELECT embeding FROM edu_data_embeddings', con=engine) 
    
    n = len(ids)
    project_embeddings = np.zeros((n, 768))
    
    for i in range(n):
        project_embeddings[i:] = json.loads(df_embeddings["embeding"].iloc[i])
    
   
    # Load the pre-trained model
    model = SentenceTransformer('stsb-mpnet-base-v2')

    # computing embedding of search_key
    search_key_embedding = model.encode(search_key, show_progress_bar=False)
    search_key_embedding = search_key_embedding.reshape(1, -1)

    # compute cosine similarity
    similarity_score = cosine_similarity(project_embeddings, search_key_embedding)

    # find ids where similarity score is larger than thresshold

    sim_score = np.append(similarity_score, ids,  axis= 1) 
    #print(sim_score)
    
    df_sim_score = pd.DataFrame(sim_score, columns = ['sim_score', 'id_project'])
    #print(df_sim_score)

    df1 = df_sim_score.sort_values(by="sim_score", ascending=False)
    related_project = df1.query('sim_score > 0.30')['id_project']
    
    return(related_project)

#semantic_search('Science Competition')

search_keys_test = ["Science Competition", "language", "encyclopedia", "database", "travel", "school", "medicine", "university", "research", "jungle", "kitchen", "rain", "elephant", "grass", "cars", "sun", "children", "chair"]

for i in search_keys_test:
    print(i)
    print(semantic_search(i))

    
    
    
# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')
    
    

Science Competition
7     3.0
0    30.0
Name: id_project, dtype: float64
language
18    17.0
20    31.0
23    34.0
Name: id_project, dtype: float64
encyclopedia
3     12.0
12     4.0
16    10.0
24    20.0
31    23.0
27    28.0
1     13.0
17     2.0
14    14.0
8      8.0
30    27.0
4     11.0
Name: id_project, dtype: float64
database
16    10.0
Name: id_project, dtype: float64
travel
Series([], Name: id_project, dtype: float64)
school
16    10.0
31    23.0
Name: id_project, dtype: float64
medicine
28    26.0
Name: id_project, dtype: float64
university
30    27.0
Name: id_project, dtype: float64
research
17     2.0
2     29.0
Name: id_project, dtype: float64
jungle
Series([], Name: id_project, dtype: float64)
kitchen
Series([], Name: id_project, dtype: float64)
rain
Series([], Name: id_project, dtype: float64)
elephant
Series([], Name: id_project, dtype: float64)
grass
Series([], Name: id_project, dtype: float64)
cars
Series([], Name: id_project, dtype: float64)
sun
Series([], Name: id_p

In [ ]:
#Execution time: 141.69155955314636 seconds
